In [1]:
import transformers
import pandas as pd
import numpy as np
import nltk
from utils import TweetsDataset
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
tokenizer = transformers.AutoTokenizer.from_pretrained("xlm-roberta-base")
df = pd.read_csv("../../data/procesed.csv")

In [3]:
model = transformers.XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=4, problem_type="multi_label_classification")

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [4]:
df_train, df_valid = TweetsDataset.split_test_val(df,test_size=0.20,valid_size=0.20)
train_data_loader = TweetsDataset.create_data_loader(df_train, tokenizer, batch_size=8)
valid_data_loader = TweetsDataset.create_data_loader(df_valid, tokenizer, batch_size=8)

In [5]:
model = model.to(device)
EPOCHS = 10
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
total_steps = len(train_data_loader) * EPOCHS
scheduler = transformers.get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0, 
    num_training_steps = total_steps
)

In [6]:
from collections import defaultdict
from utils import Model
history = defaultdict(list)
Model.train(EPOCHS, model, train_data_loader, optimizer, device, scheduler, history, len(df_train), len(df_valid), valid_data_loader)

Epoch 1/10
----------
Train loss: 0.541 Accuracy: 0.354
Validation loss 0.536 Accuracy: 0.364 F1 score: 0.364

Epoch 2/10
----------
Train loss: 0.538 Accuracy: 0.353
Validation loss 0.535 Accuracy: 0.364 F1 score: 0.364

Epoch 3/10
----------
Train loss: 0.537 Accuracy: 0.353
Validation loss 0.535 Accuracy: 0.364 F1 score: 0.364

Epoch 4/10
----------
Train loss: 0.537 Accuracy: 0.351
Validation loss 0.535 Accuracy: 0.364 F1 score: 0.364

Epoch 5/10
----------
Train loss: 0.536 Accuracy: 0.354
Validation loss 0.535 Accuracy: 0.364 F1 score: 0.364

Epoch 6/10
----------


KeyboardInterrupt: 

In [12]:
torch.save(model.state_dict(), 'xlm_roberta_base.bin')

In [14]:
# Test 
from utils import Results, Model
model = transformers.XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=4, problem_type="multi_label_classification")
model.to(device)
model.load_state_dict(torch.load('xlm_roberta_base.bin'))
df_test = pd.read_csv('../../data/development_test.csv')
df_test = pd.DataFrame({
    "tweets": df_test["tweet"],
    "labels": df_test["ideology_multiclass"].map({'moderate_left': 0, 'moderate_right': 1, "left": 2, "right": 3})
})
test_data_loader = TweetsDataset.create_data_loader(df_test, tokenizer)
acc, loss, f1 = Model.test(model, test_data_loader, device, len(df_test))
Results.add_result("xml-roberta-base", "ideology-multiclass", acc, loss, f1)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

Test loss: 0.694 Accuracy: 0.122


H:\GitHub\political-sentiment-analysis-esp\lib\huggingface\utils\Results.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(pd.DataFrame({
